|<h2>Course:</h2>|<h1><a href="https://udemy.com/course/dullms_x/?couponCode=202508" target="_blank">A deep understanding of AI language model mechanisms</a></h1>|
|-|:-:|
|<h2>Part 6:</h2>|<h1>Intervention (causal) mech interp<h1>|
|<h2>Section:</h2>|<h1>Interfering with attention <h1>|
|<h2>Lecture:</h2>|<h1><b>CodeChallenge HELPER: Does GPT2 like pineapple pizza?<b></h1>|

<br>

<h5><b>Teacher:</b> Mike X Cohen, <a href="https://sincxpress.com" target="_blank">sincxpress.com</a></h5>
<h5><b>Course URL:</b> <a href="https://udemy.com/course/dullms_x/?couponCode=202508" target="_blank">udemy.com/course/dullms_x/?couponCode=202508</a></h5>
<i>Using the code without the course may lead to confusion or errors.</i>

In [ ]:
import numpy as np
import scipy.stats as stats

from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib as mpl

from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
import torch.nn.functional as F

import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

# Exercise 1: Model, hooks, tokens

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
model = GPT2LMHeadModel.from_pretrained('gpt2-large')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

model = model.to(device)
model.eval()

In [ ]:
# some useful variables
nheads
nlayers
n_emb
head_dim

In [ ]:
def hook_silencer(layer_number):
  def hook2zero(module,input):

    # modify the activation only for this layer
    if

      # reshape so we can index heads
      head_tensor =

      # ablate
      head_tensor

      # reshape back to tensor
      head_tensor =

      # return a tuple matching the original
      input = (head_tensor,*input[1:])

    return
  return hook2zero




# Tokens (my apologies to the Italians :P )

In [ ]:
tokens = tokenizer.encode('Peanut butter and pineapple taste great on pizza',return_tensors='pt').to(device)
nbatches,ntokens = tokens.shape

for i in range(ntokens):
  print

In [ ]:
butter_idx =
pineap_idx =
pizza_idx  =

butter_idx,pineap_idx,pizza_idx

# Exercise 2: Does GPT2 prefer peanut butter or pineapple on pizza?

In [ ]:
layer2silence =
head2silence =

with torch.no_grad():


# check hidden states sizes


In [ ]:
# get cossim from all layers
cs_clean = np.zeros((nlayers,2))

# loop over layers
for layeri in range(nlayers):

  # cosine similarities
  cs_clean[layeri,0] = F.cosine_similarity(
  cs_clean[layeri,1] = F.cosine_similarity(

In [ ]:
plt.figure(figsize=(8,5))

plt.plot(
plt.plot(

plt.gca().set(xlabel='Transformer block',ylabel='Cosine similarity',
              title='Cosine similarity to " pizza"')
plt.legend()
plt.show()

# Exercise 3: Can we disrupt GPT's pizza preference?

In [ ]:
token2silence =
head2silence =

cs_manip = np.zeros((nlayers,2))

for layer2silence

  with torch.no_grad():
    out_manip =

  # cosine similarities
  cs_manip[layer2silence,0] = F.cosine_similarity
  cs_manip[layer2silence,1] = F.cosine_similarity

In [ ]:
fig,axs = plt.subplots(1,2,figsize=(12,4))

axs[0].plot(np.arange(nlayers)-.1,cs_manip[:,0],'ko',markerfacecolor=[.9,.7,.7,.6],markersize=10,label='Peanut butter')

axs[1].plot(np.arange(nlayers)-.1,cs_manip[:,0]-cs_clean[:,0],'ko',markerfacecolor=[.9,.7,.7,.6],markersize=10,label='Peanut butter')

plt.tight_layout()
plt.show()

# Exercise 4: Layer-specific silencing

In [ ]:
# redefined
token2silence =

# initializes results matrix
cs_manip = np.zeros

# loop over layers and heads
for layer2silence in tqdm(:
  for head2silence in

    # forward pass


    # cosine similarities
    cs_manip[layer2silence,head2silence,0] = F.cosine_similarity
    cs_manip[layer2silence,head2silence,1] = F.cosine_similarity

In [ ]:
fig,axs = plt.subplots(1,2,figsize=(12,4))

for i in range(nlayers):
  axs[0].plot(
  axs[0].plot(

  # run a ttest
  tres = stats.ttest_1samp



axs[0].set(xlabel='Transformer block',ylabel='Cosine similarity',title='Cosine similarity in clean and silenced models')

axs[1].set(xlabel='Transformer block',ylabel='$\Delta$ cosine similarity',title='Silenced - clean')
axs[1].axhline(0,color='k',linewidth=.8,zorder=-10)

plt.tight_layout()
plt.show()